# More Commands, Command Loop Problems, a Java IDE

Now that we have the base `Command` class and a `CommandScheduler` we can really start talk about
programming for the robot, moving from this notebook environment to a real dvelopment environment, and
start learning about project structure and the wpilib.

Before we start, we need our `Command`, `CommandSceduler`, and `CountedCommand` classes. We will paste those in the cell below, and run that cell so we have these classes available to us.

In [22]:
// This is a Command class, initially we will just start with a constructor,
// naming, and an execute method.
abstract class Command
{
    private String m_name;
    
    Command()
    {
        // By default the name is the class name. If this is called
        // from the constructor of an extending class, then the
        // class name will be the name of the extending class.
        m_name = getClass().getName();
        m_name = m_name.substring(m_name.lastIndexOf('$') + 1);
    }
    
    void initialize()
    {
        
    }
    
    Command setName(String name)
    {
        m_name = name;
        return this;
    }
    
    String getName()
    {
        return m_name;
    }
    
    void execute()
    {
        //System.out.println("executing: " + m_name);
    }
    
    boolean isFinished()
    {
        return false;
    }
    
    void end(boolean interrupted)
    {
        
    }
}

class CountedCommand extends Command 
{
    // the step count
    private long m_stepCt = 0;
    // the number of steps we run before we stop
    final long m_numberOfSteps;
    
    CountedCommand(long numberOfSteps)
    {
        super();
        m_numberOfSteps = numberOfSteps;
    }
    
    @Override
    void initialize()
    {
        m_stepCt = 0;
    }
    
    
    @Override
    void execute()
    {
        m_stepCt += 1;
        System.out.println(getName()  + ":  stepCt: " + m_stepCt + " of " + m_numberOfSteps);
    }
    @Override
    boolean isFinished()
    {
        return m_stepCt >= m_numberOfSteps;
    }
}

// create and run a command to make sure it works
Command command = new CountedCommand(2).setName("test CountedCommand");
while (!command.isFinished()) {
    command.execute();
}

System.out.println("\nReset command:");
command.initialize();
while (!command.isFinished()) {
    command.execute();
}

class CommandScheduler
{
    final long RUN_TIME;
    final long STEP_TIME;
    
    // this is the list of commands to be executed the next time commands are run.
    private final List<Command> m_commandList = new ArrayList<>();
    // this is the list of commans waiting to be scheduled the next time commands are run.
    private final List<Command> m_toBeScheduled = new ArrayList<>();

    CommandScheduler(long runTime, long stepTime)
    {
        RUN_TIME = runTime;
        STEP_TIME = stepTime;
    }
    
    void schedule(Command command)
    {
        m_toBeScheduled.add(command);
    }
    
    void run()
    {
        // Setup the timing for running the commands in the loop.
        long startTime = System.currentTimeMillis();
        System.out.println("startTime = " + startTime);
        long endTime = startTime + RUN_TIME;
        long nextStepEnd = startTime;


        while (System.currentTimeMillis() < endTime)
        {
            // control of the step time
            long sleepTime = nextStepEnd - System.currentTimeMillis();
            if (sleepTime > 0)
            {
                try {
                    Thread.sleep(sleepTime);
                } catch(InterruptedException e) {
                    // OK, something is trying to kill this program,
                    // break out of the loop.
                    break;
                }
            }
            System.out.println("time = " + System.currentTimeMillis() + "; wanted to run at: " + nextStepEnd);
            nextStepEnd += STEP_TIME;
            
            // check the currently scheduled commands and remove any that have finished
            Iterator<Command> iter = m_commandList.iterator();
            while (iter.hasNext()) {
                Command command = iter.next();
                if (command.isFinished()) {
                    // remove the command we just got with the call to next()
                    iter.remove();
                    // call the end() method so the command can finish, the argument is false
                    // because the command was not interupted, but ended as expected.
                    command.end(false);
                }
            }
            
            // Adding any commands wating for scheduling
            iter = m_toBeScheduled.iterator();
            while (iter.hasNext()) {
            // get the next command to be scheduled
                Command command = iter.next();
                // remove it from the m_toBeScheduled list
                iter.remove();
                // initialize it
                command.initialize();
                // and add it to the scheduled command list
                m_commandList.add(command);
            }


            // run the commands
            iter = m_commandList.iterator();
            while (iter.hasNext()) 
            {
                iter.next().execute();
            }
        }
    }
}

CommandScheduler commandScheduler = new CommandScheduler(400, 40);
Command countedCommand4 = new CountedCommand(4);
commandScheduler.schedule(new CountedCommand(10).setName("cmd1"));
commandScheduler.schedule(new CountedCommand(5).setName("cmd2"));
commandScheduler.schedule(countedCommand4);
commandScheduler.run();


System.out.println("\nReschedule countedCommand4 and run the command scheduler");
commandScheduler.schedule(countedCommand4);
commandScheduler.run();

test CountedCommand:  stepCt: 1 of 2
test CountedCommand:  stepCt: 2 of 2

Reset command:
test CountedCommand:  stepCt: 1 of 2
test CountedCommand:  stepCt: 2 of 2
startTime = 1627064006733
time = 1627064006734; wanted to run at: 1627064006733
cmd1:  stepCt: 1 of 10
cmd2:  stepCt: 1 of 5
CountedCommand:  stepCt: 1 of 4
time = 1627064006777; wanted to run at: 1627064006773
cmd1:  stepCt: 2 of 10
cmd2:  stepCt: 2 of 5
CountedCommand:  stepCt: 2 of 4
time = 1627064006817; wanted to run at: 1627064006813
cmd1:  stepCt: 3 of 10
cmd2:  stepCt: 3 of 5
CountedCommand:  stepCt: 3 of 4
time = 1627064006853; wanted to run at: 1627064006853
cmd1:  stepCt: 4 of 10
cmd2:  stepCt: 4 of 5
CountedCommand:  stepCt: 4 of 4
time = 1627064006893; wanted to run at: 1627064006893
cmd1:  stepCt: 5 of 10
cmd2:  stepCt: 5 of 5
time = 1627064006934; wanted to run at: 1627064006933
cmd1:  stepCt: 6 of 10
time = 1627064006976; wanted to run at: 1627064006973
cmd1:  stepCt: 7 of 10
time = 1627064007018; wanted to r

# Making Some Cool New Commands

We have a pretty complete command infrastructure here. In this section we will talk about making a couple useful commands based on the `CountedCommand`. This will show you a little about implementing a bit of useful
functionality in a base class `CountedCommand` you can use many other places.

## Shooter Lift Command

This parallels our 2020-2021 season command to lift the ball into our shooter. The deal here is that we want to
enable the lift when the command is initialized (in the competition robot, this happened when a shoot button was pushed). The command extended the lift for some period of time (number of command steps) that it took the lift to reach full extension and deliver the ball to the shooter rollers - then the lift retracted so the next ball could position itself for lift. This is the code for a shooter lift that extends the `CountedCommand` class:

In [11]:
class ShooterLift extends CountedCommand
{
    boolean m_shooterLiftEnabled = false;
    
    ShooterLift(long numberOfSteps)
    {
        super(numberOfSteps);
    }
    
    @Override
    void initialize()
    {
        super.initialize();
        m_shooterLiftEnabled = true;
        // code to enable shooter lift subsystem
        System.out.println("Extend Lift");
    }
    
    @Override
    void end(boolean interupted)
    {
        m_shooterLiftEnabled = false;
        // code to disable shooter lift subsystem
        System.out.println("Retract Lift");
    }   
}

CommandScheduler commandScheduler = new CommandScheduler(400, 40);
commandScheduler.schedule(new ShooterLift(3));
commandScheduler.run();


startTime = 1626977098984
Extend Lift
time = 1626977098984; wanted: 1626977098984
ShooterLift:  stepCt: 1 of 3
time = 1626977099028; wanted: 1626977099024
ShooterLift:  stepCt: 2 of 3
time = 1626977099066; wanted: 1626977099064
ShooterLift:  stepCt: 3 of 3
time = 1626977099109; wanted: 1626977099104
Retract Lift
time = 1626977099149; wanted: 1626977099144
time = 1626977099185; wanted: 1626977099184
time = 1626977099228; wanted: 1626977099224
time = 1626977099266; wanted: 1626977099264
time = 1626977099307; wanted: 1626977099304
time = 1626977099346; wanted: 1626977099344
time = 1626977099386; wanted: 1626977099384


OK, not much code - why is this so simple? `CountedCommand` does the heavy lifting here. it knows how to
run a command for some number of steps and we extend that functionality. In this case, the only things we
need to do with this command are:
* extend `initialize()` to enable the lift;
* extend `end(boolean interupted)` to disable the lift.


## Delayed Command

In [23]:
class DelayedCommand extends CountedCommand
{
    final Command m_nextCommand;
    final CommandScheduler m_commandScheduler;
    
    DelayedCommand(long numberOfSteps, Command command, CommandScheduler commandScheduler)
    {
        super(numberOfSteps);
        m_nextCommand = command;
        m_commandScheduler = commandScheduler;
    }
    
    @Override
    void end(boolean interupted)
    {
        m_commandScheduler.schedule(m_nextCommand);
        System.out.println("Scedule: " + m_nextCommand.getName());
    }
}

// Testn this command
CommandScheduler commandScheduler = new CommandScheduler(1000, 40);
commandScheduler.schedule(new DelayedCommand(6, new CountedCommand(3), commandScheduler));
commandScheduler.run();


startTime = 1627064017138
time = 1627064017138; wanted to run at: 1627064017138
DelayedCommand:  stepCt: 1 of 6
time = 1627064017178; wanted to run at: 1627064017178
DelayedCommand:  stepCt: 2 of 6
time = 1627064017220; wanted to run at: 1627064017218
DelayedCommand:  stepCt: 3 of 6
time = 1627064017258; wanted to run at: 1627064017258
DelayedCommand:  stepCt: 4 of 6
time = 1627064017301; wanted to run at: 1627064017298
DelayedCommand:  stepCt: 5 of 6
time = 1627064017339; wanted to run at: 1627064017338
DelayedCommand:  stepCt: 6 of 6
time = 1627064017378; wanted to run at: 1627064017378
Scedule: CountedCommand
CountedCommand:  stepCt: 1 of 3
time = 1627064017418; wanted to run at: 1627064017418
CountedCommand:  stepCt: 2 of 3
time = 1627064017460; wanted to run at: 1627064017458
CountedCommand:  stepCt: 3 of 3
time = 1627064017499; wanted to run at: 1627064017498
time = 1627064017539; wanted to run at: 1627064017538
time = 1627064017579; wanted to run at: 1627064017578
time = 1627064

## Using the `ShooterLift` and `DelayedCommand` Together

In [24]:
CommandScheduler commandScheduler = new CommandScheduler(1000, 40);
ShooterLift shooterLift = new ShooterLift(3);
commandScheduler.schedule(shooterLift);
commandScheduler.schedule(new DelayedCommand(9, shooterLift, commandScheduler));
commandScheduler.schedule(new DelayedCommand(18, shooterLift, commandScheduler));
commandScheduler.run();

startTime = 1627064023830
time = 1627064023830; wanted to run at: 1627064023830
Extend Lift
ShooterLift:  stepCt: 1 of 3
DelayedCommand:  stepCt: 1 of 9
DelayedCommand:  stepCt: 1 of 18
time = 1627064023874; wanted to run at: 1627064023870
ShooterLift:  stepCt: 2 of 3
DelayedCommand:  stepCt: 2 of 9
DelayedCommand:  stepCt: 2 of 18
time = 1627064023911; wanted to run at: 1627064023910
ShooterLift:  stepCt: 3 of 3
DelayedCommand:  stepCt: 3 of 9
DelayedCommand:  stepCt: 3 of 18
time = 1627064023950; wanted to run at: 1627064023950
Retract Lift
DelayedCommand:  stepCt: 4 of 9
DelayedCommand:  stepCt: 4 of 18
time = 1627064023994; wanted to run at: 1627064023990
DelayedCommand:  stepCt: 5 of 9
DelayedCommand:  stepCt: 5 of 18
time = 1627064024033; wanted to run at: 1627064024030
DelayedCommand:  stepCt: 6 of 9
DelayedCommand:  stepCt: 6 of 18
time = 1627064024070; wanted to run at: 1627064024070
DelayedCommand:  stepCt: 7 of 9
DelayedCommand:  stepCt: 7 of 18
time = 1627064024112; wanted 

# What Happens if we put a Sleep or Loop in a Command?

# Moving to IDEA